## Wind Modeling 

The wind model in erad is built using typical hurricane data. The model can be imported using the command below

{cite}`williamstsunami2020`
{cite}`bennettextending2021`

In [1]:
from erad.models.hazard import WindModel

An instance of `WindModel` requires the following pieces of information,
- **timestamp**: The timestamp for the hurricane event
- **center**: Center of the hurricane in lat / long format
- **max_wind_speed**: Maximum wind speed
- **air_pressure**: Air pressuse of the hurricame
- **radius_of_max_wind**: Radius of the maximum wind speed
- **radius_of_closest_isobar**: Radius of closest isobar

In [2]:
from datetime import datetime

from gdm.quantities import Distance
from shapely.geometry import Point

from erad.quantities import Speed, Pressure

hurricane = WindModel(
    name = 'hurricane 1',
    timestamp=datetime.now(),
    center=Point(87, 20),
    max_wind_speed=Speed(20, "miles/hour"),
    air_pressure=Pressure(10000, "hPa"),
    radius_of_max_wind=Distance(30, "miles"),
    radius_of_closest_isobar=Distance(100, "miles"),
)
hurricane.pprint()

WindModel(
    name='hurricane 1',
    timestamp=datetime.datetime(2025, 5, 8, 8, 52, 28, 80390),
    center=<POINT (87 20)>,
    max_wind_speed=<Quantity(20, 'mile / hour')>,
    radius_of_max_wind=<Quantity(30, 'mile')>,
    radius_of_closest_isobar=<Quantity(100, 'mile')>,
    air_pressure=<Quantity(10000, 'hectopascal')>
)

An example of the `WindModel` can be built using the `example()` methods for testing purposes.

In [3]:
earthquake = WindModel.example()
earthquake.pprint()


WindModel(
    name='hurricane 1',
    timestamp=datetime.datetime(2025, 5, 8, 8, 52, 28, 357580),
    center=<POINT (-121.93 36.601)>,
    max_wind_speed=<Quantity(50, 'mile / hour')>,
    radius_of_max_wind=<Quantity(50, 'mile')>,
    radius_of_closest_isobar=<Quantity(300, 'mile')>,
    air_pressure=<Quantity(1013.25, 'hectopascal')>
)

## Building from historical events

Erad allows users to build wind models from historic hurricane events as well. The `from_hurricane_sid` class method can be used to build wind models representing historic events. 

In [4]:
hurricane_track = WindModel.from_hurricane_sid('2017228N14314')
for h in hurricane_track:
    h.pprint()
    break
print(f"number of hurricane points : {len(hurricane_track)}")

WindModel(
    name='2017228N14314',
    timestamp=Timestamp('2017-08-16 06:00:00'),
    center=<POINT (13.7 -45.8)>,
    max_wind_speed=<Quantity(25.0, 'knot')>,
    radius_of_max_wind=<Quantity(80.0, 'nautical_mile')>,
    radius_of_closest_isobar=<Quantity(150.0, 'nautical_mile')>,
    air_pressure=<Quantity(1013.0, 'millibar')>
)

number of hurricane points : 141


## Plotting the Wind Model


In [33]:
from collections import defaultdict
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd

data  = defaultdict(list)
for h in hurricane_track:
    data["timestamp"].append(h.timestamp)
    data["lat"].append(h.center.x)
    data["lon"].append(h.center.y)
    data["air_pressure"].append(h.air_pressure.magnitude)
    data["max_wind_speed"].append(h.max_wind_speed.magnitude)
    data["radius_of_max_wind"].append(h.radius_of_max_wind.magnitude)
    data["radius_of_closest_isobar"].append(h.radius_of_closest_isobar.magnitude)


data = pd.DataFrame(data)

fig = go.Figure()

fig.add_trace(go.Scattergeo(
    lat=data["lat"],
    lon=data["lon"],
    mode='markers',
    marker=dict(
        size=data["radius_of_closest_isobar"]/5,
        color="lightblue",
        opacity=0.4
    ),
    name='Radius of closest isobar'  # Name for the legend
))
fig.add_trace(go.Scattergeo(
    lat=data["lat"],
    lon=data["lon"],
    mode='markers',
    marker=dict(
        size=data["radius_of_max_wind"]/5,
        color=data["max_wind_speed"],
        colorbar=dict(
            title='Maximum win speed (knot)',
            orientation='h', # Set orientation to horizontal
            x=0.5,           # Center the color bar horizontally
            y=-0.25 
            ), # Add color bar title
        colorscale='Plasma',
        opacity=0.4
    ),
    name='Radius of max wind'  # Name for the legend
))
fig.show()